In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import pickle
from plothist import make_hist, plot_hist

In [ ]:
# Define dummy classifier ouptut

N = 10000

signal = np.random.normal(1, 0.25, N)
signal = signal[(signal>0) & (signal<1)]
background = np.random.normal(0, 0.25, N)
background = background[(background>0) & (background<1)]

y_score = np.concatenate((signal,background))
y_true = np.concatenate((np.ones(signal.shape, dtype=bool), np.zeros(background.shape, dtype=bool)))
y_weight = np.ones(len(signal)+len(background))

In [ ]:
fig, ax = plt.subplots()
h_sig = make_hist(y_score[y_true], bins=20, range=(0, 1), weights=y_weight[y_true])
h_bkg = make_hist(y_score[~y_true], bins=20, range=(0, 1), weights=y_weight[~y_true])
plot_hist(h_sig, ax=ax, label="Signal", histtype="step", linewidth=1.2)
plot_hist(h_bkg, ax=ax, label="Background", histtype="step", linewidth=1.2)
ax.set_xlabel('Classifier ouput')
ax.set_ylabel("Entries")
ax.legend()
fig.savefig("classifier_output.svg", bbox_inches="tight")

In [ ]:
# Sort according to score
desc_score_indices = np.argsort(y_score, kind="stable")[::-1]
y_score = y_score[desc_score_indices]
y_true = y_true[desc_score_indices]
y_weight = y_weight[desc_score_indices]

In [ ]:
tps = np.cumsum(y_true * y_weight)
efficiency = tps/np.sum(y_true * y_weight)

# translate classifier selection into a signal efficiency with an interpolation
# fit only 1 point out of 10 to save memory
# You may want to take more points if the sample is small
fit_efficiency = interp1d(y_score[::-1][::10], efficiency[::-1][::10], fill_value=(1.0, 0.0), bounds_error=False)

# The fit can be saved for later use
# with open("efficiency.pickle"), 'wb') as handle:
#   pickle.dump(eff, handle)

In [ ]:
fig, ax = plt.subplots()
ax.plot(y_score, efficiency, ".", label='Expectation')
ax.plot(y_score, fit_efficiency(y_score), '--', label='Fit')
ax.set_xlabel('Classifier ouput')
ax.set_ylabel('Signal efficiency')
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.legend()

In [ ]:
fig, ax = plt.subplots()
h_sig_fit = make_hist(fit_efficiency(y_score[y_true]), bins=20, range=(0, 1), weights=y_weight[y_true])
h_bkg_fit = make_hist(fit_efficiency(y_score[~y_true]), bins=20, range=(0, 1), weights=y_weight[~y_true])
plot_hist(h_sig_fit, ax=ax, label="Signal", histtype="step", density=True, linewidth=1.2)
plot_hist(h_bkg_fit, ax=ax, label="Background", histtype="step", density=True, linewidth=1.2)
ax.set_xlabel('Signal efficiency')
ax.legend()

In [ ]:
fig, ax = plt.subplots()
h_sig_fit_rej = make_hist(1-fit_efficiency(y_score[y_true]), bins=20, range=(0, 1), weights=y_weight[y_true])
h_bkg_fit_rej = make_hist(1-fit_efficiency(y_score[~y_true]), bins=20, range=(0, 1), weights=y_weight[~y_true])
plot_hist(h_sig_fit_rej, ax=ax, label="Signal", histtype="step", density=True, linewidth=1.2)
plot_hist(h_bkg_fit_rej, ax=ax, label="Background", histtype="step", density=True, linewidth=1.2)
ax.set_xlabel('1 $-$ Signal efficiency')
ax.set_ylabel("Entries")
ax.legend()
fig.savefig("flatten_distribution.svg", bbox_inches="tight")